In [1]:
import pandas as pd
import numpy as np

In [2]:
cs_df = pd.read_csv("cumulative-stats.tsv", sep='\t')
pd_df = pd.read_json("player-lookup.json", orient='index')
print(cs_df.shape)
print(pd_df.shape)

(3571, 9)
(340, 6)


In [3]:
full_df = pd_df.merge(cs_df, on='player_id', how='inner')
print(full_df.shape)

(3571, 14)


In [4]:
full_df['age_year'] = full_df['age_start'] + (full_df['year'] - full_df['year_start'])

In [5]:
full_df

,player_id,player_name,age_start,year_start,year_end,last_name,year,yards,tds,att,comp,playoff_games,playoff_wins,pb_games,age_year
0,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1989,1749,9,293,155,0,0,0,23
1,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1990,4328,20,692,381,0,0,0,24
2,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1991,7082,31,1055,618,0,0,1,25
3,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1992,10527,54,1528,920,3,3,2,26
4,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1993,13627,69,1920,1191,6,6,3,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3566,ZornJi00,Jim Zorn,23,1976,1987,Zorn,1983,20042,107,2973,1586,0,0,0,30
3567,ZornJi00,Jim Zorn,23,1976,1987,Zorn,1984,20122,107,2990,1593,0,0,0,31
3568,ZornJi00,Jim Zorn,23,1976,1987,Zorn,1985,20916,111,3113,1649,0,0,0,32
3569,ZornJi00,Jim Zorn,23,1976,1987,Zorn,1986,20916,111,3113,1649,0,0,0,33


In [6]:
max_year = full_df.groupby(['player_id'])['year_end'].max()
max_wins = full_df.groupby(['player_id'])['playoff_wins'].max()
max_age = full_df.groupby(['player_id'])['age_year'].max()

In [7]:
full_df = full_df.merge(max_year, on='player_id', how='inner')
full_df = full_df.merge(max_wins, on='player_id', how='inner')
full_df = full_df.merge(max_age, on='player_id', how='inner')

In [8]:
full_df.rename(columns = {"playoff_wins_x": "playoff_wins",
                          "year_end_x": "year_end",
                          "year_end_y": "max_year", 
                          "playoff_wins_y": "max_wins",
                          "age_year_x": "age_year",
                          "age_year_y": "max_age"}, inplace = True)

full_df = full_df[full_df['max_wins'] > 0]

full_df.head()

,player_id,player_name,age_start,year_start,year_end,last_name,year,yards,tds,att,comp,playoff_games,playoff_wins,pb_games,age_year,max_year,max_wins,max_age
0,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1989,1749,9,293,155,0,0,0,23,2000,11,34
1,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1990,4328,20,692,381,0,0,0,24,2000,11,34
2,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1991,7082,31,1055,618,0,0,1,25,2000,11,34
3,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1992,10527,54,1528,920,3,3,2,26,2000,11,34
4,AikmTr00,Troy Aikman,23,1989,2000,Aikman,1993,13627,69,1920,1191,6,6,3,27,2000,11,34


In [9]:
full_df.to_csv(r'full-stats.csv', index = True)